In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from collections import defaultdict
from datetime import date

import pandas as pd
from tqdm.notebook import tqdm
from bs4 import Tag

from polimi_scraper.config import DataPath
from polimi_scraper.utils import occupancy_soup

In [5]:
soup = occupancy_soup("MIA", date(2025, 2, 3))
soup

<!DOCTYPE html>

<html class="no-js">
<head>
<title>Occupazioni per Data</title>
<link href="https://webcommons.polimi.it/webcommons/assets/ateneo2014.css.jsp?v=5&amp;lang=it&amp;dt_version=1.10" rel="stylesheet" type="text/css"/>
<link href="https://webcommons.polimi.it/webcommons/ajax/libs/jqueryui/1.12.1/themes/polij.css.jsp?v=5&amp;lang=it" rel="stylesheet" type="text/css"/>
<link href="https://webcommons.polimi.it/webcommons/assets/desktop.css.jsp?v=5&amp;lang=it" rel="stylesheet" type="text/css"/>
<link href="/spazi/assets/css/base.css?__load_ts=1737140432539" rel="stylesheet" type="text/css"/>
<script src="https://webcommons.polimi.it/webcommons/ajax/libs/jqueryui/1.12.1/ui.js.jsp?v=5&amp;lang=it&amp;dt_version=1.10" type="text/javascript"></script>
<script src="https://webcommons.polimi.it/webcommons/ajax/libs/desktop.js.jsp?v=5&amp;lang=it" type="text/javascript"></script>
<script type="text/javascript">
	$(document).ready(function(){
		$('#loading').remove();
	});
</script>
<

In [6]:
classrooms: list[Tag] = soup.find_all("td", class_="dove")
classrooms

[<td class="dove" rowspan="1">
 <a href="Aula.do?evn_init=espandi&amp;idaula=32" title="Milano Città Studi - Piazza Leonardo da Vinci 32 - Edificio 2 - Piano Terra"> 2.0.1  </a>
 </td>,
 <td class="dove" rowspan="1">
 <a href="Aula.do?evn_init=espandi&amp;idaula=33" title="Milano Città Studi - Piazza Leonardo da Vinci 32 - Edificio 2 - Piano Primo"> 2.0.2  </a>
 </td>,
 <td class="dove" rowspan="1">
 <a href="Aula.do?evn_init=espandi&amp;idaula=34" title="Milano Città Studi - Piazza Leonardo da Vinci 32 - Edificio 2 - Piano Primo"> 2.1.1  </a>
 </td>,
 <td class="dove" rowspan="1">
 <a href="Aula.do?evn_init=espandi&amp;idaula=35" title="Milano Città Studi - Piazza Leonardo da Vinci 32 - Edificio 2 - Piano Primo"> 2.1.2  </a>
 </td>,
 <td class="dove" rowspan="1">
 <a href="Aula.do?evn_init=espandi&amp;idaula=36" title="Milano Città Studi - Piazza Leonardo da Vinci 32 - Edificio 2 - Piano Primo"> 2.1.3  </a>
 </td>,
 <td class="dove" rowspan="1">
 <a href="Aula.do?evn_init=espandi&amp;

In [7]:
classroom_free = defaultdict(list)

for classroom in tqdm(classrooms, desc="Room"):
    classroom_name = classroom.find("a").string.strip()
    for cell in classroom.find_next_siblings("td"):
        cell: Tag
        if "slot" in cell.attrs.get("class", []):
            classroom_free[classroom_name].extend([False] * int(cell.attrs["colspan"]))
        else:
            classroom_free[classroom_name].append(True)

classroom_free

Room:   0%|          | 0/154 [00:00<?, ?it/s]

defaultdict(list,
            {'2.0.1': [False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              False,
              True,
              True,
              True,
              True,
           

In [12]:
pd.DataFrame(classroom_free).to_parquet(DataPath.OCCUPANCY)